## Задание:

Сегодня мы сделаем функцию по поиску дифференциально экспрессированных генов при помощи критерия Манна-Уитни-Уилкоксона (кратко - Wilcox rank-sum test). Именно этот тест используют в большинстве статей по сравнению дифференциальной экспрессии генов или когда вы не знаете распределение ваших данных.

После получения p-values мы также сделаем поправку на множественные сравнения, что обязательно для любой публикации.

### Пример того, как пользоваться U тестом

In [3]:
import numpy as np
from scipy import stats

# Two independent samples
group1 = np.array([8, 12, 15, 17, 22, 25, 30])
group2 = np.array([5, 10, 12, 18, 20, 23, 27])

# Perform the Wilcoxon rank-sum (Mann-Whitney U) test
stat, p_value = stats.mannwhitneyu(group1, group2, alternative='two-sided')

print(f"U-statistic: {stat}")
print(f"P-value: {p_value.round(2)}")

U-statistic: 27.5
P-value: 0.75


### Примените этот тест к каждому гену

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests

In [2]:
from io import BytesIO

# Замените ссылку на вашу публичную ссылку Яндекс.Диска
public_url = 'https://disk.yandex.ru/d/ldXeauf7K9qC9A'

# Получение прямой ссылки на скачивание
download_url = requests.get(
    'https://cloud-api.yandex.net/v1/disk/public/resources/download',
    params={'public_key': public_url}
).json()['href']

# Скачивание файла
file_response = requests.get(download_url)

# Чтение содержимого в датафрейм
df = pd.read_csv(BytesIO(file_response.content))

# Вывод датафрейма
print(df)

     Unnamed: 0      TMCC1     RANBP3    GABRG3     ARRDC5       LRP3  \
0             0  33.888550  38.384340  0.044721   1.385159  11.341853   
1             1  16.497124  35.054090  0.314645   4.329797   8.649321   
2             2   5.449933  31.783958  0.002949   2.137536   1.750892   
3             3  33.362240  56.147026  0.037295   1.604904   6.807299   
4             4  50.188510  30.165264  0.039678   1.501439   1.619954   
..          ...        ...        ...       ...        ...        ...   
995         495  11.731787  23.128685  0.100596  12.412588   0.605454   
996         496   9.942467  33.065037  0.000000   6.337149   4.170284   
997         497  28.450361  41.614952  0.012397  14.773923   6.238087   
998         498   9.897264  35.510582  0.000000   3.920270   2.957491   
999         499  13.769289  26.868212  0.194126   4.888445   2.344540   

         TIMM23      TBPL1     BIRC8     TTC28  ...     MAGEA6       IL4I1  \
0    105.193306  26.463990  0.000000  1.53787

In [3]:
# Для скорости
expression_data = df.set_index(df.Cell_type)

In [4]:
b_cells_expression_data = expression_data.loc['B_cell']
nk_cells_expression_data = expression_data.loc['NK_cell']

In [5]:
nk_cells_expression_data.head()

,Unnamed: 0,TMCC1,RANBP3,GABRG3,ARRDC5,LRP3,TIMM23,TBPL1,BIRC8,TTC28,...,MAGEA6,IL4I1,LCN12,SMG6,C1orf100,WDR75,ZBTB26,SPTY2D1,PLEKHA2,Cell_type
Cell_type,,,,,,,,,,,,,,,,,,,,,
NK_cell,0,18.431755,43.209717,0.134517,17.764618,8.714558,37.152515,31.299683,0.0,1.809592,...,31.865282,0.082100,2.66793,18.149836,0.000000,34.555800,11.242084,9.979048,56.609380,NK_cell
NK_cell,1,17.316593,44.242010,0.108825,2.882164,32.696762,47.112465,14.575365,0.0,4.718271,...,11.912389,3.564077,0.00000,27.930525,0.001398,26.036766,3.961336,12.515713,44.365517,NK_cell
NK_cell,2,16.492771,49.170765,0.349678,8.010749,27.401628,79.537470,38.602650,0.0,5.906733,...,5.850225,0.107140,0.00000,26.257713,0.003091,33.604733,6.964660,14.168297,96.972786,NK_cell
NK_cell,3,13.640349,51.758680,0.099209,6.262011,4.809357,50.365715,35.139390,0.0,2.232243,...,0.042824,1.480726,0.00000,17.019278,0.000000,35.286602,8.747441,9.610327,57.677250,NK_cell
NK_cell,4,23.893585,47.919067,0.190005,17.054922,8.314115,49.035330,37.189670,0.0,5.157692,...,9.800081,21.140158,0.00000,17.292538,0.000000,26.834440,7.560658,12.403855,78.070790,NK_cell


Ваша имплементация здесь

In [17]:
b_cells_expression_data = df[df['Cell_type'] == 'B_cell'].reset_index(drop=True)
nk_cells_expression_data = df[df['Cell_type'] == 'NK_cell'].reset_index(drop=True)

def diff_expr_genes():
    genes_exp = []
    num_columns = df.shape[1]

    for i in range(num_columns - 1):
        col_name = df.columns[i]

        for m in range(min(b_cells_expression_data.shape[0], nk_cells_expression_data.shape[0])):
            genes_diff = b_cells_expression_data.iloc[m, i] - nk_cells_expression_data.iloc[m, i]
            genes_exp.append(genes_diff)

        if len(genes_exp) > 0:
            genes_exp_mean = sum(genes_exp) / len(genes_exp)
      #      print(f"Mean difference for column '{col_name}': {genes_exp_mean.round(2)}")
            genes_exp.clear()
        else:
            print(f"No data available to compute difference for column '{col_name}'.")

# Call the function
diff_expr_genes()

### Найдя p-values, их нужно поправить на множественные сравнения.

Используя пример, примените это для ваших p-values

In [10]:
import numpy as np
from statsmodels.stats.multitest import multipletests

# Example p-values from multiple tests
p_values = np.array([0.01, 0.04, 0.03, 0.06, 0.02, 0.20, 0.15])

# Apply FDR correction (Benjamini-Hochberg)
reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

print(f"Reject Null Hypothesis: {reject}")
print(f"Corrected p-values: {pvals_corrected}")

Reject Null Hypothesis: [False False False False False False False]
Corrected p-values: [0.07  0.07  0.07  0.084 0.07  0.2   0.175]


In [16]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests

# Предполагается, что 'df' — ваш DataFrame
b_cells_expression_data = df[df['Cell_type'] == 'B_cell'].reset_index(drop=True)
nk_cells_expression_data = df[df['Cell_type'] == 'NK_cell'].reset_index(drop=True)

def diff_expr_genes():
    gene_names = df.columns[:-1]  # Исключаем столбец 'Cell_type'
    p_values = []
    gene_diff = {}

    for gene in gene_names:
        # Извлекаем значения экспрессии гена
        b_expr = b_cells_expression_data[gene]
        nk_expr = nk_cells_expression_data[gene]

        # Выполняем t-тест
        stat, p_value = ttest_ind(b_expr, nk_expr, equal_var=False)
        p_values.append(p_value)

    # Преобразуем p-значения в массив NumPy
    p_values = np.array(p_values)

    # Применяем корректировку Бенджамини-Хохберга
    reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')


    # Выводим результаты
    for i, gene in enumerate(gene_names):
        print(f"Ген: {gene}")
        print(f"  Оригинальное p-значение: {p_values[i]:.4f}")
        print(f"  Скорректированное p-значение: {pvals_corrected[i]:.4f}")
        print(f"  Значимо после корректировки: {'Да' if reject[i] else 'Нет'}\n")
        if reject[i]:
            gene_diff[gene] = pvals_corrected[i].round(2)
    for gene, pvals_corrected in gene_diff.items():
      print(f'Gene: {gene}')
      print(f'Correctes p values: {pvals_corrected.round(3)}\n')
   # print(gene_diff)

# Вызываем функцию
diff_expr_genes()

Streaming output truncated to the last 5000 lines.
Correctes p values: 0.03

Gene: TCL1A
Correctes p values: 0.0

Gene: GEM
Correctes p values: 0.0

Gene: ABCC10
Correctes p values: 0.0

Gene: KCTD9
Correctes p values: 0.0

Gene: GPRC5D
Correctes p values: 0.0

Gene: RNF168
Correctes p values: 0.02

Gene: PLEKHO1
Correctes p values: 0.0

Gene: IFNA4
Correctes p values: 0.0

Gene: DDX41
Correctes p values: 0.0

Gene: LPXN
Correctes p values: 0.0

Gene: TRPV2
Correctes p values: 0.0

Gene: XRCC1
Correctes p values: 0.0

Gene: CBFA2T2
Correctes p values: 0.0

Gene: GFER
Correctes p values: 0.0

Gene: SNW1
Correctes p values: 0.0

Gene: EZR
Correctes p values: 0.0

Gene: NME8
Correctes p values: 0.03

Gene: RCAN3
Correctes p values: 0.0

Gene: ZNF264
Correctes p values: 0.0

Gene: SIDT2
Correctes p values: 0.0

Gene: C6orf89
Correctes p values: 0.0

Gene: TBXAS1
Correctes p values: 0.0

Gene: NEMP1
Correctes p values: 0.0

Gene: ACO2
Correctes p values: 0.02

Gene: SPPL2A
Correctes p value